In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
y = train.SalePrice
X = train.drop('SalePrice', axis = 1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, random_state = 0, test_size = 0.2)

categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype == 'object' and X_train_full[cname].nunique() < 10]
categorical_cols

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]
numerical_cols

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [4]:
X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
618,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
870,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
92,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
817,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
302,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


# Step 1: Define Preprocessing Steps

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numerical_transformer = SimpleImputer(strategy = 'constant')

categorical_transformer = Pipeline(steps = [('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers = [('num', numerical_transformer, numerical_cols), ('cat', categorical_transformer, categorical_cols)])

# Step 2: Define the Model

In [6]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 100, random_state = 0)

# Step 3: Create and Evaluate the Pipeline

In [8]:
from sklearn.metrics import mean_absolute_error

my_pipeline = Pipeline(steps = [('preprocessor', preprocessor),
                               ('model', model)])

my_pipeline.fit(X_train, y_train)
y_pred = my_pipeline.predict(X_valid)

score = mean_absolute_error(y_valid, y_pred)
score

17740.290308219177